# BAŞLIK

Let's create random word set!

In [6]:
# TODO: KAYDEDİLEN DİCT'TE SADECE TOKEN LİSTLERİ OLSA YETERLİ, TOKEN LEN GRUPLARI ZATEN KEY'LER ARACILIĞI İLE ERİŞİLEBİLİR + OLARAK HER GRUBUN LENLERİ KAYDEDİLEBİLİR
# TODO: çince arapça vs bu alfabeleri de temizlemeli!

### Random Word Set Oluşturma Adımları

1. **Kelimeleri Bulma**  
   Girilen metin içerisinde, kelimeleri bulmalıyım. Bunu `tokenizer.normalize_str` ve `tokenizer.pre_tokenize_str` kullanarak gerçekleştireceğim ki böylece tokenizer pipeline'nına bağlı kalalım. (sayı ve punctiation karakterlerini random olarak koymak istemediğimden random word set'ten atacağım)

2. **Tokenizasyon**  
   Bulunan kelimeler, tokenize edilmelidir.

3. **Statistiksel Analiz ve Temizleme**  
   Nadir kelimeler atılmalı. Daha sonra token_len'e göre gruplaştırma yapılmalı. Bu seferde gruplara göre stat bakılmalı: 3 tokenlık kaç kelime var vs gibi. Noktalama, sayılar, semboller de atılmalı!

4. **Son Temsil**  
   Son temsil aşağıdaki formatta olmalıdır:
   ```plaintext
   (token_size: 5, 5_tokenlık_kelime_listesi_tokenized_btw, word_count: 5644) 
   ```
   - `word_count` atılmamalı, rastgele indeks alırken kullanılacaktır.

5. **Dosya Kaydetme**  
   Temizlenmiş veri, bir JSON dosyası olarak kaydedilmelidir.


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# standard library
import os
import sys
import re
import itertools
import multiprocessing as mp
from collections import Counter

# third party library
import tqdm
import pandas as pd


# local library
import bert_implementation_tr.data.old_data as old_data
import random_word_set


ModuleNotFoundError: No module named 'bert_implementation_tr'

In [9]:
slow_tokenizer = random_word_set.slow_tokenizer
fast_tokenizer = random_word_set.fast_tokenizer



LIMIT_FOR_TOKEN_GROUP = 5                   # 5 ten fazla token'a sahip kelimeleri istemiyoruz (token_len <= 5)
MAX_WORD_LIMIT_FOR_TOKEN_GROUP = 5_000      # gruplarda da maximum belirtilen sayıda kelime olabilecek
MIN_FREQ_FOR_WORDS = 150                    # gruplarda az occur olan kelimelerin atılmasını gerekiyor (çince, arapça vs)


In [10]:
if __name__ == "__main__":

    
    files = ["raw/" + file for file in os.listdir("raw")]
    # to able to use mp pool
    merged_file_content = old_data.get_merged_files(files=files)

    # TODO dev time arttırmak için slice yapacağım
    merged_file_content = merged_file_content.splitlines()
    len_merged_file_content = len(merged_file_content)

                                   

    # Create a multiprocessing pool
    with mp.Pool(old_data.NUM_PROCESSES) as pool:
        merged_file_content = pool.imap(random_word_set.normalize_line, merged_file_content, chunksize= 2048)
        merged_file_content = list(tqdm.tqdm(merged_file_content, total=len_merged_file_content, desc="[INFO] Normalizing lines..."))    

    # Create a multiprocessing pool
    with mp.Pool(old_data.NUM_PROCESSES) as pool:
        merged_file_content = pool.imap(random_word_set.pre_tokenize_line, merged_file_content, chunksize= 2048)
        merged_file_content = list(tqdm.tqdm(merged_file_content, total=len_merged_file_content, desc="[INFO] Pre-tokenizing lines..."))    
    


    # list[list[str]] -> list[str]
    merged_file_content = list(itertools.chain.from_iterable(merged_file_content))

    print(f"[INFO] Counting words...")
    frequency = Counter(merged_file_content)
    most_common = frequency.most_common(50) 

    frequency_df = pd.DataFrame(frequency.items(), columns=["word", "frequency"])

    # free memory
    del merged_file_content
    del frequency 


    # New columns for token_ids and token_len
    frequency_df["token_ids"]=None
    frequency_df["token_len"]=None

    
    # Create a multiprocessing pool
    with mp.Pool(old_data.NUM_PROCESSES) as pool:
        iterable_freq_df = pool.imap(random_word_set.tokenize_word, frequency_df.iterrows(), chunksize= 216)
        frequency_df = list(tqdm.tqdm(iterable_freq_df, total=len(frequency_df), desc="[INFO] Tokenization of words..."))  
        frequency_df = pd.DataFrame(frequency_df, columns=["word", "frequency", "token_ids", "token_len"])



    print(f"[INFO] Most common 50 words: {most_common}")
    print(f"[INFO] Total number of unique words: {len(frequency_df)}")
    print(f"[INFO] Total number of all words: {frequency_df['frequency'].sum()}")


    group_list = []
    
    for group_name, group_df in frequency_df.groupby("token_len"):
        
        # bundan büyük token grupları ile işimiz yok
        if int(group_name) > LIMIT_FOR_TOKEN_GROUP:
            continue

        
        # rasgele sampling yapmadan önce grupta belli bir frequency altındaki kelimeleri atmalıyız
        group_df = group_df[group_df["frequency"] >= MIN_FREQ_FOR_WORDS]


        number_of_sample = len(group_df) if len(group_df) < MAX_WORD_LIMIT_FOR_TOKEN_GROUP else MAX_WORD_LIMIT_FOR_TOKEN_GROUP


        # her gruptan belirtilen sayıda sampling yapılacak
        group_list.append(group_df.sample(n=number_of_sample, random_state=13013))


    del frequency_df


    print(f"[INFO] Saving random_word_set.json file... ")
    
    total_df = pd.concat(group_list)
    total_df.to_json("random_word_set.json", orient="records", lines=True, force_ascii=False)





    

TypeError: get_merged_files() got an unexpected keyword argument 'files'

In [ ]:
{'doc': 'Valongo Portekiz\'de aynı adı taşıyan Valongo belediyesine bağlı olan, 20,66\xa0km² yüzölçümüne sahip, 18.698 nüfuslu (2001) bir bucaktır (Portekizce: "freguesia")', 'token_ids': [3501, 3853, 1020, 4618, 11, 1960, 2413, 2382, 5054, 3501, 3853, 1020, 10877, 2366, 2065, 
16, 22, 20, 16, 26, 26, 2449, 22, 10255, 2427, 16, 21, 28, 18, 26, 29, 28, 9455, 12, 22, 20, 20, 21, 13, 1941, 13106, 12, 9940, 30, 6, 11812, 6, 13], 'word_ids': [0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43], 'sent_idx': [47]}
{'doc': "Benoit Richaud (16 Ocak, 1988; Avignon, Fransa) Fransız buz patenci. 2008 yılında Terra Findlay ile birlikte kaymaya başlamıştır. 
Çift 2008-2009 ISU Junior Grand Prix çerçevesinden Courchevel, Fransa'da düzenlenen yarışmada 4. olmuştur.\nRichaud 2005-2007 yılları arasında Elodie Brouiller ile birlikte çalışmıştır. Takım 2006 Fransa Gençler şampiyonu ve 2007 Fransa Gençler ikincisi olmuştur. Çift iki kez Dünya Gençler Artistik Patinaj Şampiyonalarına katılmış ve 2007 Dünya Gençlerde 7. olmuştur.\nBrouiller ile kaymaya başlamadan önce Scarlett Rouzet ile çalışmıştır. 2008 yılında Terra Findlay ile birlikte bir takım oluşturdu ve çift ilk yarışmalarına 2008-2009 ISU Junior Grand Prix çerçevesinde Fransa'da yapılan yarışmaya katıldılar. Çift zorunlu dansı ilk sırada bitirdi. Orijinal dans sırasında ise düştüler ve bu 
bölümü 11. sırada tamamladılar. Serbest dansda 3. olarak genel toplamda 4. sıraya yükseldiler", 'token_ids': [2566, 16720, 5530, 5275, 1018, 12, 21, 26, 2797, 16, 21, 29, 28, 28, 31, 27508, 7994, 1936, 16, 3224, 13, 3142, 4541, 28169, 18, 22, 20, 20, 28, 2090, 30690, 2603, 29897, 1946, 2023, 2484, 20692, 1975, 4161, 18, 3447, 22, 20, 20, 28, 17, 22, 20, 20, 29, 24480, 15348, 6116, 8073, 9844, 1008, 21945, 5328, 7691, 16, 3224, 11, 1964, 4040, 8277, 24, 18, 2695, 18, 5530, 5275, 1018, 22, 20, 20, 25, 17, 22, 20, 20, 27, 2966, 2277, 2328, 10998, 1007, 
4491, 14564, 1937, 2023, 2484, 7953, 18, 2265, 22, 20, 20, 26, 3224, 7194, 5620, 1933, 22, 20, 20, 27, 3224, 7194, 10421, 2695, 18, 3447, 2319, 2766, 2458, 7194, 13776, 17697, 12608, 2453, 9175, 1933, 22, 20, 20, 27, 2458, 7194, 1934, 27, 18, 2695, 18, 4491, 14564, 1937, 2023, 
20692, 1975, 14208, 2523, 18831, 20512, 2440, 2149, 1948, 2023, 7953, 18, 22, 20, 20, 28, 2090, 30690, 2603, 29897, 1946, 2023, 2484, 1941, 2265, 7324, 1933, 3447, 2155, 19025, 2391, 22, 20, 20, 28, 17, 22, 20, 20, 29, 24480, 15348, 6116, 8073, 9844, 3224, 11, 1964, 2632, 10040, 20955, 18, 3447, 7993, 16729, 2155, 3376, 6450, 18, 5019, 5161, 2949, 2219, 6432, 1937, 1933, 1989, 3804, 21, 21, 18, 3376, 4966, 1927, 18, 4122, 5161, 1931, 23, 18, 2013, 2330, 5851, 24, 18, 9474, 5751, 1937], 'word_ids': [0, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 12, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 26, 26, 27, 28, 29, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 
43, 44, 45, 46, 46, 47, 47, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 58, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 70, 70, 71, 71, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 106, 107, 108, 109, 109, 110, 111, 112, 113, 114, 114, 114, 115, 116, 116, 117, 118, 119, 119, 120, 120, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 130, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 172, 173, 174, 175, 176, 177, 178, 179, 180, 180, 181, 182, 183, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 192], 'sent_idx': [24, 39, 68, 92, 
110, 148, 188, 195, 211, 224]}

In [13]:
random_word_set_dict["token_len_group_4"], random_word_set_dict["token_len_group_4_length"]

([[3107, 7027, 15038, 1926],
  [25230, 1008, 1030, 1934],
  [23164, 10465, 17465, 1011],
  [18175, 9338, 2789, 2140],
  [5578, 1948, 4270, 27026],
  [158, 1033, 1178, 2223],
  [151, 2501, 4481, 2381],
  [2318, 3485, 9907, 5217],
  [24599, 21665, 3332, 2002],
  [20321, 12643, 1005, 2085],
  [3248, 2046, 19699, 10331],
  [6377, 11545, 22538, 1011],
  [14248, 4844, 2222, 6184],
  [7792, 2091, 2988, 5931],
  [4180, 2276, 7587, 1016],
  [8062, 4496, 2102, 8722],
  [7856, 1009, 13621, 2327],
  [14737, 20650, 1005, 1009],
  [8150, 3776, 2994, 2577],
  [3971, 2134, 9539, 7329],
  [2676, 17408, 10634, 2022],
  [4140, 4112, 6302, 2022],
  [13450, 1921, 16228, 1019],
  [6583, 25002, 4895, 3221],
  [4334, 3837, 2536, 1963],
  [2603, 4197, 3250, 3570],
  [97, 2063, 8444, 1008],
  [17986, 22538, 4012, 1016],
  [2569, 20964, 18327, 1011],
  [8812, 7587, 10861, 1025],
  [18259, 2537, 5245, 1027],
  [48, 2223, 18135, 1020],
  [3592, 18121, 1201, 1030],
  [11368, 1078, 2456, 1924],
  [3191, 23630, 4109,

In [4]:
pd.DataFrame.from_dict(random_word_set_dict["token_len_group_1"])

,0
0,16975
1,5859
2,12659
3,11148
4,21297
...,...
4995,14260
4996,12878
4997,18603
4998,24641


In [17]:
pd.DataFrame.from_dict(random_word_set_dict["token_len_group_1"]) # old

,word,token_ids,token_len
22069,olaylara,[16975],1
8732,listesinde,[5859],1
22035,anlayışı,[12659],1
6085,vatandaşı,[11148],1
16226,şekliyle,[21297],1
...,...,...,...
23343,gordon,[14260],1
32856,desteklenen,[12878],1
4013,eva,[18603],1
7249,bastırmak,[24641],1


In [5]:
frequency_df.head(10)

,word,frequency,token_ids,token_len
0,=,47238,[32],1
1,thomas,119,[6647],1
2,lüthi,2,"[13229, 3255]",2
3,(,20570,[12],1
4,d,2524,[44],1
5,.,155540,[18],1
6,6,15960,[26],1
7,eylül,1304,[2873],1
8,1,68283,[21],1
9,9,37906,[29],1


In [5]:
frequency_df[["token_len", "frequency"]].groupby("token_len").sum()

,frequency
token_len,
1,241632
2,29205
3,8298
4,1995
5,395
6,112
7,29
8,9
9,8


In [6]:
frequency_df[["token_len", "frequency"]].groupby("token_len").sum() # after deleting numbers and punctiations from text

,frequency
token_len,
1,143386
2,32060
3,9562
4,2490
5,668
6,195
7,48
8,19
9,11


In [16]:
frequency_df[["token_len", "frequency"]].groupby("token_len").sum()

,frequency
token_len,
1,2443197
2,286093
3,79524
4,17962
5,3970
6,931
7,318
8,148
9,78


token_len 5 ve öncesini tutacağım

In [22]:
deneme = frequency_df.head(25)
deneme

,word,frequency,token_ids,token_len
0,=,47238,[32],1
1,thomas,119,[6647],1
2,lüthi,2,"[13229, 3255]",2
3,(,20570,[12],1
4,d,2524,[44],1
5,.,155540,[18],1
6,6,15960,[26],1
7,eylül,1304,[2873],1
8,1,68283,[21],1
9,9,37906,[29],1


In [32]:
deneme.sample(n= (len(deneme) if len(deneme) < 30 else 30), random_state=13013)

,word,frequency,token_ids,token_len
22,prix,75,[8073],1
17,moto,3,"[3543, 1020]",2
7,eylül,1304,[2873],1
0,=,47238,[32],1
14,",",113641,[16],1
18,2,44964,[22],1
21,grand,209,[6116],1
23,dünya,2445,[2458],1
20,için,10517,[2051],1
8,1,68283,[21],1


In [31]:
import re

text = "Hello, World! 123 5 5 5555? ~~'^ ^ #         aa */*/-+"

# Find punctuation
punctuations = re.findall(r'[^\w\s]', text)
print("Punctuation marks:", punctuations)

# Find numbers
numbers = re.findall(r'\d+', text)
print("Numbers:", numbers)


Punctuation marks: [',', '!', '?', '~', '~', "'", '^', '^', '#', '*', '/', '*', '/', '-', '+']
Numbers: ['123', '5', '5', '5555']


In [32]:
normalizer = normalizers.Sequence([normalizers.NFKC(),
                                       normalizers.Lowercase(),
                                       normalizers.Replace(Regex('[^\w\s]'),""),   # for numbers
                                       normalizers.Replace(Regex('\d+'),"") ])     # for punctiations

In [33]:
normalizer.normalize_str(text)

'hello world                aa '

In [ ]:
total_df.to_json("random_word_set.json", orient="records", lines=True, force_ascii=False)

In [4]:
a = pd.read_json("random_word_set.json", orient="records", lines=True, encoding="utf-8")

In [5]:
a.head(10)

,word,frequency,token_ids,token_len
0,çadır,415,[14594],1
1,temelde,995,[17473],1
2,roy,884,[13952],1
3,oluştuğunu,634,[23155],1
4,reklam,3040,[6580],1
5,vektör,1135,[9327],1
6,faust,268,[21706],1
7,projesi,4675,[6874],1
8,orta,33740,[2673],1
9,suçsuz,329,[31200],1


In [54]:
a.sort_values(by="frequency", ascending=False).iloc[10_500]

word           göstermişti
frequency              250
token_ids    [15099, 2008]
token_len                2
Name: 6302, dtype: object

In [15]:
temp_dic = {}

for name, grup in a.groupby("token_len"):
    temp_dic[f"token_group_{name}"] = grup["token_ids"].to_list()
    temp_dic[f"token_group_{name}_length"] = len(temp_dic[f"token_group_{name}"])

In [16]:
temp_dic.keys()

dict_keys(['token_group_1', 'token_group_1_length', 'token_group_2', 'token_group_2_length', 'token_group_3', 'token_group_3_length', 'token_group_4', 'token_group_4_length', 'token_group_5', 'token_group_5_length'])

In [30]:
temp_dic["token_group_3"][:5]

[[17227, 22538, 1011],
 [9857, 22538, 1921],
 [14665, 22538, 1011],
 [27939, 22538, 1007],
 [7707, 22538, 1011]]

In [36]:
def get_random_word_set():
    if not os.path.exists("random_word_set.json"):
        print(f"[INFO] random_word_set.json is not already exists. Try to execute random_word_set.py script to generate this file before calling this function...")
        exit(0)
    
    random_word_set_df = pd.read_json("random_word_set.json", orient="records", lines=True, encoding="utf-8")
    random_word_set_dict = {}
    
    for group_name, group in random_word_set_df.groupby("token_len"):
        random_word_set_dict[f"token_group_{group_name}"] = group["token_ids"].to_list()
        random_word_set_dict[f"token_group_{group_name}_length"] = len(random_word_set_dict[f"token_group_{group_name}"])
    
    return random_word_set_dict

In [39]:
bc = get_random_word_set()

In [41]:
bc["token_group_3"][:5], bc.keys()

([[17227, 22538, 1011],
  [9857, 22538, 1921],
  [14665, 22538, 1011],
  [27939, 22538, 1007],
  [7707, 22538, 1011]],
 dict_keys(['token_group_1', 'token_group_1_length', 'token_group_2', 'token_group_2_length', 'token_group_3', 'token_group_3_length', 'token_group_4', 'token_group_4_length', 'token_group_5', 'token_group_5_length']))

# STAGE0 + STAGE1


The purpose of this script is to create the data for the BERT training

NSP taski büyük veri setlerinde aslında avantajlı değil aksine model performansını düşürüyor (electra paper'i sanırsam buna bak)
Ancak küçük veri setleri için kullanımı (ki benim durumum) genel model performansını olumlu yönde etkiler.
NSP taskininde düzgün çalışabilmesi için A sonu, B başının gerçek cümlelerdeki gibi olması önemlidir.
Diğer türlü A cümle ortası ile biter, B de cümle ortası ile başlarsa (A: Ali ata (bak) B: (Sevdiğim) şarkılar bunlar..., notNext)
Model nsp taskinde bu tarz example'ları basit gramatik bilgiler ile (ki genelde modelin ilk öğreneceği şeylerdir) halledebilir, 
complex ilişkilendirmelere gerek duymaz.

görünüşe göre pad token'ına ihtiyacım olmayacak.

(., !, ?, ..., :, ;) cümleleri bu tokenlara göre split edeceğim.

"kelime" açısından istenilen oranlar sağlansa da, span corruption yapıldığından "token" açısından
istatistiği merak ediyorum yüzdesel olarak ne kadar artacak?

toplam veri seti default overlap param ile yaklaşık 2 x artacak (overlap yarı old'dan 0,5 x gelecek, notNext old yani random sample
olduğunda kernel/block hareketi/posizyonu esasında değişmeyecek bunu da havadan yeni bir example oluşturma gibi düşünebilirsin + 0,5 x
daha toplamda yaklaşık olarak 2x artmış olur) 

tokenların yaklaşık yarısı bir epoch'da 3 kere gözükecek (overlap + random sample sayesinde)
kalan yarısı da 2 kere gözükecek (sadece overlap sayesinde) 

random word set stage:
    * bunu script şekilde yazdım. burada (data.py) eğer beklenen dosya yok ise bu exec edilsin
    * word_count_per_token_len_group (örn 500 ise her token grubu için en fazla 500 tane kelime olacak )
    * 
    * 
    *


Her stage hali hazırda yapılmış ise atlansın
Her stage kalınıldığı noktadan devam edebilmeli (shard index ile)
1st Stage(documents):
    * read trwiki-67 files and merge them (as a big one string object)
    * split titles and docs (just keep docs btw, i will not use titles for training)(boş/null doc olmamalı)
    * tüm docs list shuffle edilsin (raw dosya dizilimi random şekilde mi bilmiyorum, ondan her türlü bir shuffling yapalım)
    * delete subtitles from docs (aproximately if line has less than 4 words) (list of docs)
    * save this list of docs json in shards
    * bu stage'in düzgün çalışıp çalışmadığını kontrol et
    * free all resources

Her stage hali hazırda yapılmış ise atlansın
Her stage kalınıldığı noktadan devam edebilmeli (shard index ile)
2nd Stage (ab_tokenized):
    * 2.1 Stage (tokenization)
        + load docs json file as pandas dataframe (fonk yaz belirtilen shard indexini yükleyen)
        + en son kalınılan shard'tan devam edebilmeli
        + her doc "string" tokenize edilecek yeni kolonlar türeyecek: token_ids, word_ids
        + düz string kolonu dropla (list[str]) olan
        + her doc'da sentence indexlerini bulmalı yeni kolona bunları koymalı (doc uzunluğu en sonki index'mi öyle değil ise ekle)
        + kısaca token len için ayrı kolon olmayacak, sentence index'lerdeki en son elemandan bu çıkartılabilmeli
        + 2 cümleden az olan (yani tek cümleli) doc'lar filtrelenecek
        + doc [token_ids, word_ids, sent_idx]
        + bu stage'in düzgün çalışıp çalışmadığını kontrol et
        + 2.2'ye geçmeden free resources

    * 2.2 Stage (ab creation)
        + block_size, overlap, shard başına kaç ab sample (shard_size) gibi parametrelere göre çalışacak
        + isnext olasılığı al, next ise:
            - bloğu doldurulabildiği kadar doldurulacak, yer kalırsa (truncation sayesinde çoğu durumda kalmasa da document'ın sonuna
              geldiğimizde dolduramama ihtimali var) [PAD] token'ları ile doldurulacak
        + notNext ise:
            - random sample alınacak, sample genişliği belli kısıtlamalar ile olacak, eğer kısıtlamalar sağlanamıyorsa başka random sample
              yapılacak
            - random sample alındıktan sonra bulunulan kernel/block posizyonunda gene alınabilecek kadar alınacak (sonu cümle gibi bitmeli) [çok düşük ihtimal ama ilk sentence aşarsa bir sonraki iterasyona atlanmalı]
            - kalan yerler pad ile doldurulacak (bunda pad olasılığı yüksek)
        + tüm shard shuffle edilmeli (ab halde shuffle et shard'ı)
        + ab sample'ında kaç tane kelime var ise ayrı kolon yap
        + 2.stage sonu, shard kayıtları (ab sayılarına göre yapılabilir (her doc farklı sayıda ab çıkarabilir sonuçta) ancak aynı kalsın, zaten son
          aşamada her shard'ı token sayısı eşit olacak biçimde tasarlayacağım (o aşamaya kadar şard'larımız hep aynı sayıda doc olacak şekilde))
        + bu stage'in düzgün çalışıp çalışmadığını kontrol et
        + free all resources

        ++ toplam veri seti default overlap param ile yaklaşık 2 x artacak (overlap yarı old'dan 0,5 x gelecek, notNext old yani random sample
           olduğunda kernel/block hareketi/posizyonu esasında değişmeyecek bunu da havadan yeni bir example oluşturma gibi düşünebilirsin + 0,5 x
           daha toplamda yaklaşık olarak 2x artmış olur) 
           tokenların yaklaşık yarısı bir epoch'da 3 kere gözükecek (overlap + random sample sayesinde)
        ++ kalan yarısı da 2 kere gözükecek (sadece overlap sayesinde) 
    

Her stage hali hazırda yapılmış ise atlansın
Her stage kalınıldığı noktadan devam edebilmeli (shard index ile)
3rd Stage(xy_numpy):
    * xy'ler numpy arrayler vs oluşturma, SON aşama
    * % kaç [MASK] token'i var?, % kaç yer değiştirme, aynı bırakma var vs ("kelime" açısından istenilen oranlar sağlansa da, span corruption yapıldığından "token" açısından
    istatistiği merak ediyorum yüzdesel olarak ne kadar artacak?) 
    * ekstrem kelimelerde (8 token misal) random kelime yoksa kaçsın ya da MASK yapsın
    * random word json, unpackleneerek sadece list[list[int]] şekline çevirip kullanmayı deneyelim (idx'ler token len)
      böylece manager dict yerine düz Array kullanabiliriz
    * random kelimeleri negative sample gibi düşünebiliriz

In [68]:
# standard library
import os
import sys
import json
import logging
import random
import multiprocessing as mp
from typing import List


# third party library
import tqdm
import pandas as pd
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast

# local
import bert_implementation_tr.data.old_data as old_data
random.seed(13013)


In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
NUM_SHARDS_FOR_DOC = 100
NUM_DOCS_PER_SHARD = 0
NUM_SHARDS_FOR_AB = 100
NUM_SHARDS_FOR_XY = 100

In [211]:
def delete_subtitles_from_docs(docs):
    """
    Yaklaşıl 4 kelimeden oluşan cümleler hep alt başlık gibi (wiki dump kaynaklı). Veriye bakarken bu alt başlıkların paragraflar arasında anlam bozukluğuna sebep olabileceğini düşündüm
    Küçük veri seti kullanacak olan modellerde bu sinyallerin performansı kötü etkileyeceğini düşündüğümden (deney yapılabilir)
    bunları silmeye karar kıldım.
    """
    new_docs = []
    for doc in docs:
        new_doc = []
        for line in doc.splitlines():
            if len(line.split(" ")) > 5:
                new_doc.append(line)
        if new_doc: # for example, doc can have 1 sentence and it has less then 5 words
            # let's convert in the same format (join lines again)
            new_docs.append("\n".join(new_doc))
    
    
    assert not any(len(doc) == 0 for doc in new_docs), "[INFO] Unexpected, empty docs list has empty doc list..."

    return new_docs

def get_docs_df():
    
    print(f"[INFO] Docs dataframe is being created...")

    merged_content = old_data.get_merged_files()
    _, docs = old_data.split_titles_and_docs(merged_content)

    del merged_content

    # inplace operation
    random.shuffle(docs)

    docs = delete_subtitles_from_docs(docs)

    return pd.DataFrame(docs, columns=["doc"])

def create_doc_shards(docs_df):

    global NUM_SHARDS_FOR_DOC
    global NUM_DOCS_PER_SHARD

    NUM_DOCS_PER_SHARD = len(docs_df) // NUM_SHARDS_FOR_DOC


    if len(docs_df) % NUM_SHARDS_FOR_DOC != 0:
        extra_shard_len = len(docs_df) % NUM_SHARDS_FOR_DOC
        NUM_SHARDS_FOR_DOC += 1
        


    os.makedirs("doc_shards", exist_ok=True)
    files = os.listdir("./doc_shards")


    if len(files) == NUM_SHARDS_FOR_DOC:
        print(f"[INFO] Expected number of shards are already created. Exiting...")
        return
    
    # fazlalık dosyaları (eğer olursa) sil (evet biraz brute force)
    if NUM_SHARDS_FOR_DOC < len(files):

        files_finded = set(files)
        shard_files_expected = {f"doc_shard_{shard_idx}.json" for shard_idx in range(NUM_SHARDS_FOR_DOC)}
        files_will_deleted = list(files_finded - shard_files_expected)
        if files_will_deleted:
            print(f"[INFO] These files are unexpected, so they are gonna be deleted: {files_will_deleted}")
            for file in files:
                os.remove("./doc_shards/" + file)


    print(f"[INFO] Total number of doc: {len(docs_df)}")
    print(f"[INFO] Each shard will have {NUM_DOCS_PER_SHARD} number of docs...")
    print(f"[INFO] Number of shard will be: {NUM_SHARDS_FOR_DOC}")
    if extra_shard_len:
        print(f"[INFO] Extra shard needed. Number of doc will be placed: {extra_shard_len}")

    
    df_start_idx = 0
    df_end_idx = NUM_DOCS_PER_SHARD


    with tqdm.tqdm(total=NUM_SHARDS_FOR_DOC, desc="Doc shards are being created") as pbar:
        for shard_idx in range(0, NUM_SHARDS_FOR_DOC):
            # last iteration (extra shard)
            if shard_idx == NUM_SHARDS_FOR_DOC - 1:
                df_end_idx = df_start_idx + extra_shard_len

                assert df_end_idx == len(docs_df)
                assert (df_end_idx - df_start_idx) == extra_shard_len

                docs_df.iloc[df_start_idx:df_end_idx].to_json("./doc_shards/" + f"doc_shard_{shard_idx}.json",
                                                          orient="records",
                                                          lines=True,
                                                          force_ascii=False)
                pbar.update()
                break

            docs_df.iloc[df_start_idx:df_end_idx].to_json("./doc_shards/" + f"doc_shard_{shard_idx}.json",
                                                          orient="records",
                                                          lines=True,
                                                          force_ascii=False)
            df_start_idx = df_end_idx
            df_end_idx += NUM_DOCS_PER_SHARD
            pbar.update()

def read_shard(shard_dir, shard_idx):

    if shard_idx >= 0 and shard_idx <= NUM_SHARDS_FOR_DOC:

        return pd.read_json(shard_dir + f"doc_shard_{shard_idx}.json",
                            orient="records",
                            lines=True,
                            encoding="utf-8")
    
    else:
        raise IndexError("shard_idx >= 0 and shard_idx < NUM_SHARDS_FOR_DOC")



In [72]:
# MAIN BLOCK

old_data.appy_seed()

docs_df = get_docs_df()

create_doc_shards(docs_df)

# free resource
del docs_df


[INFO] Docs dataframe is being created...
[INFO] Files in dir: ['trwiki-67-test.raw', 'trwiki-67-train.raw', 'trwiki-67-val.raw']...
[INFO] Expected number of shards are already created. Exiting...


# 2.Stage

Her stage hali hazırda yapılmış ise atlansın
Her stage kalınıldığı noktadan devam edebilmeli (shard index ile)
2nd Stage (ab_tokenized):
    * 2.1 Stage (tokenization)
        + load docs json file as pandas dataframe (fonk yaz belirtilen shard indexini yükleyen)
        + en son kalınılan shard'tan devam edebilmeli
        + her doc "string" tokenize edilecek yeni kolonlar türeyecek: token_ids, word_ids
        + düz string kolonu dropla (list[str]) olan
        + her doc'da sentence indexlerini bulmalı yeni kolona bunları koymalı (doc uzunluğu en sonki index'mi öyle değil ise ekle)
        + kısaca token len için ayrı kolon olmayacak, sentence index'lerdeki en son elemandan bu çıkartılabilmeli
        + 2 cümleden az olan (yani tek cümleli) doc'lar filtrelenecek
        + doc [token_ids, word_ids, sent_idx]
        + bu stage'in düzgün çalışıp çalışmadığını kontrol et
        + 2.2'ye geçmeden free resources

    * 2.2 Stage (ab creation)
        + block_size, overlap, shard başına kaç ab sample (shard_size) gibi parametrelere göre çalışacak
        + isnext olasılığı al, next ise:
            - bloğu doldurulabildiği kadar doldurulacak, yer kalırsa (truncation sayesinde çoğu durumda kalmasa da document'ın sonuna
              geldiğimizde dolduramama ihtimali var) [PAD] token'ları ile doldurulacak
        + notNext ise:
            - random sample alınacak, sample genişliği belli kısıtlamalar ile olacak, eğer kısıtlamalar sağlanamıyorsa başka random sample
              yapılacak
            - random sample alındıktan sonra bulunulan kernel/block posizyonunda gene alınabilecek kadar alınacak (sonu cümle gibi bitmeli) [çok düşük ihtimal ama ilk sentence aşarsa bir sonraki iterasyona atlanmalı]
            - kalan yerler pad ile doldurulacak (bunda pad olasılığı yüksek)
        + tüm shard shuffle edilmeli (ab halde shuffle et shard'ı)
        + ab sample'ında kaç tane kelime var ise ayrı kolon yap
        + 2.stage sonu, shard kayıtları (ab sayılarına göre yapılabilir (her doc farklı sayıda ab çıkarabilir sonuçta) ancak aynı kalsın, zaten son
          aşamada her shard'ı token sayısı eşit olacak biçimde tasarlayacağım (o aşamaya kadar şard'larımız hep aynı sayıda doc olacak şekilde))
        + bu stage'in düzgün çalışıp çalışmadığını kontrol et
        + free all resources

        ++ toplam veri seti default overlap param ile yaklaşık 2 x artacak (overlap yarı old'dan 0,5 x gelecek, notNext old yani random sample
           olduğunda kernel/block hareketi/posizyonu esasında değişmeyecek bunu da havadan yeni bir example oluşturma gibi düşünebilirsin + 0,5 x
           daha toplamda yaklaşık olarak 2x artmış olur) 
           tokenların yaklaşık yarısı bir epoch'da 3 kere gözükecek (overlap + random sample sayesinde)
        ++ kalan yarısı da 2 kere gözükecek (sadece overlap sayesinde) 

##### 2.1 Stage (tokenization)

- load docs json file as pandas dataframe (fonk yaz belirtilen shard indexini yükleyen)
- her doc "string" tokenize edilecek yeni kolonlar türeyecek: `token_ids`, `word_ids`
- düz string kolonu dropla (list[str] olan)
- her doc'da sentence indexlerini bulmalı yeni kolona bunları koymalı (doc uzunluğu en sonki index'mi öyle değil ise ekle)
- kısaca token len için ayrı kolon olmayacak, sentence index'lerdeki en son elemandan bu çıkartılabilmeli
- 2 cümleden az olan (yani tek cümleli) doc'lar filtrelenecek
- doc `[token_ids, word_ids, sent_idx]`
- bu stage'in düzgün çalışıp çalışmadığını kontrol et
- 2.2'ye geçmeden free resources


In [2]:
# standard library
import os
import sys
import json
import logging
import random
from typing import List, Dict
from itertools import chain
import multiprocessing as mp



# third party library
import tqdm
import spacy
import pandas as pd
import numpy as np
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast

# local
import bert_implementation_tr.data.old_data as old_data

random.seed(13013)

%load_ext autoreload
%autoreload 2

c:\Users\user\anaconda3\envs\build_and_train_bert_model\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
# DEFAULT VALUES
NUM_SHARDS_FOR_DOC = 100
NUM_DOCS_PER_SHARD = 0
NUM_SHARDS_FOR_AB = 100
NUM_SHARDS_FOR_XY = 100

# DEFAULT VALUES (değiştirilebilir/config/param)
BLOCK_SIZE = 256
OVERLAP = BLOCK_SIZE // 2 # pencere hareketi/deltası kesinlikle 4 dene!

TAMPON = 10  # block penceresi sınırlarını tamponluyoruz gibi düşünülebilir (tampon içinde kalan sent idx'ler kullanılmayacak)

tokenizer = old_data.get_tokenizer(fast=True)

# multi language sentence tokenizer
sent_seperator = spacy.load("xx_sent_ud_sm")

c:\Users\user\anaconda3\envs\build_and_train_bert_model\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
def read_doc_shard(shard_idx):
    " returns: pd.DataFrame"

    if shard_idx >= 0 and shard_idx < NUM_SHARDS_FOR_DOC:

        return pd.read_json("./doc_shards/" + f"doc_shard_{shard_idx}.json",
                            orient="records",
                            lines=True,
                            encoding="utf-8")
    
    else:
        raise IndexError("shard_idx >= 0 and shard_idx < NUM_SHARDS_FOR_DOC")

In [5]:
temp_df = read_doc_shard(0)
temp_df.columns, len(temp_df)

(Index(['doc'], dtype='object'), 3936)

In [6]:
one_doc = temp_df.iloc[13]
one_doc["token_ids"] = None
one_doc["word_ids"] = None
one_doc["sent_idx"] = None
type(one_doc), one_doc


(pandas.core.series.Series,
 doc          Sırp Demokratik Partisi (Sırpça: Српска демокр...
 token_ids                                                 None
 word_ids                                                  None
 sent_idx                                                  None
 Name: 13, dtype: object)

In [7]:
def tokenize_and_sent_idx(doc: pd.Series)-> pd.Series:
    idx = []

    temp = {"doc":doc["doc"], "token_ids":[], "word_ids":[], "sent_idx":[]}

    old_idx = 0
    last_word_id = -1
    
    for sentence in list(sent_seperator(temp["doc"]).sents):
        encoding = tokenizer(sentence.text)
        old_idx = len(encoding["input_ids"]) + old_idx
        
        temp["sent_idx"].append(old_idx)
        temp["token_ids"].extend(encoding["input_ids"])
        temp["word_ids"].extend((np.array(encoding.word_ids()) + last_word_id + 1).tolist())
        last_word_id = temp["word_ids"][-1]

    # because of zero index system
    temp["sent_idx"] = [ idx - 1 for idx in temp["sent_idx"]]

    return temp


In [8]:
tokenize_and_sent_idx(one_doc)

{'doc': 'Sırp Demokratik Partisi (Sırpça: Српска демократска странка/"Srpska demokratska stranka", СДС/SDS) Bosna-Hersek\'te bir Sırp siyasi partisidir. Şu anki lideri Mirko Šarović\'tir.\nEkim 2006\'daki parlamento seçimlerinde SDP, Sırp Cumhuriyeti Başbakanı Milorad Dodik liderliğindeki Bağımsız Sosyal Demokratlar İttifakı\'na, Sırp Cumhuriyeti\'nin önde gelen partisi ve Bosna Hersek\'teki ana Sırp partisi konumunu kaybetti. 2010 ve 2014 seçimlerinde küçük kazanımlar elde etmesine rağmen, 2018 yılına gelindiğinde partinin parlamentoda temsil oranı %20\'nin altına düştü ve tarihindeki en düşük sandalye sayısına sahip oldu. Sırp Demokratik Partisi, "savaş suçu zanlılarını tutuklayıp uluslararası bir mahkemeye teslim edemediği" gerekçesiyle ABD\'nin yaptırımı altında bulunuyor. Yaptırımlar, Amerika Birleşik Devletleri\'nden SDP\'ye herhangi bir fon ve materyal transferini yasaklar ve bunun tersi durumu için de geçerlidir. Parti, Yabancı Varlıkları Kontrol Ofisi ABD ajansı tarafından Öze

In [9]:
tokenizer.decode(tokenize_and_sent_idx(one_doc)["token_ids"][178])

'.'

In [10]:
len(tokenize_and_sent_idx(one_doc)["word_ids"]), tokenize_and_sent_idx(one_doc)["sent_idx"][-1] + 1

(490, 490)

In [11]:
a = [1,2,3,4]
[55] + a[:-1]

[55, 1, 2, 3]

In [12]:
three_one_doc_df = pd.concat([one_doc.to_frame().T, one_doc.to_frame().T, one_doc.to_frame().T])

In [13]:
three_one_doc_df

,doc,token_ids,word_ids,sent_idx
13,Sırp Demokratik Partisi (Sırpça: Српска демокр...,None,None,None
13,Sırp Demokratik Partisi (Sırpça: Српска демокр...,None,None,None
13,Sırp Demokratik Partisi (Sırpça: Српска демокр...,None,None,None


In [14]:
print(three_one_doc_df.to_dict("list"))
three_one_doc_dict = three_one_doc_df.to_dict("list")

{'doc': ['Sırp Demokratik Partisi (Sırpça: Српска демократска странка/"Srpska demokratska stranka", СДС/SDS) Bosna-Hersek\'te bir Sırp siyasi partisidir. Şu anki lideri Mirko Šarović\'tir.\nEkim 2006\'daki parlamento seçimlerinde SDP, Sırp Cumhuriyeti Başbakanı Milorad Dodik liderliğindeki Bağımsız Sosyal Demokratlar İttifakı\'na, Sırp Cumhuriyeti\'nin önde gelen partisi ve Bosna Hersek\'teki ana Sırp partisi konumunu kaybetti. 2010 ve 2014 seçimlerinde küçük kazanımlar elde etmesine rağmen, 2018 yılına gelindiğinde partinin parlamentoda temsil oranı %20\'nin altına düştü ve tarihindeki en düşük sandalye sayısına sahip oldu. Sırp Demokratik Partisi, "savaş suçu zanlılarını tutuklayıp uluslararası bir mahkemeye teslim edemediği" gerekçesiyle ABD\'nin yaptırımı altında bulunuyor. Yaptırımlar, Amerika Birleşik Devletleri\'nden SDP\'ye herhangi bir fon ve materyal transferini yasaklar ve bunun tersi durumu için de geçerlidir. Parti, Yabancı Varlıkları Kontrol Ofisi ABD ajansı tarafından Öz

In [36]:
def _get_random_sample(docs: pd.DataFrame | dict[str, list], len_of_random_b):
    
    random_b_dict = {"token_ids":[], "word_ids":[]}

    if len_of_random_b <= 0:
        raise ValueError(f"[ERROR] Len of number cannot be less than or equal to zero! {len_of_random_b} ")

    # rasgele tüm doclara gidilecek (without replacement şekilde)
    for rand_doc_idx in random.sample(range(len(docs["doc"])), k=len(docs["doc"])):
        # en sonki nokta/sent_idx gereksiz: doc sonu random sample penceremin başı olamayacağına göre
        # diğer durumların aksine bu sefer doc başı kullanışlı olabilir: random sample pencerem doc başından başlayabilir
        doc_sent_idx_list = [0] + docs["sent_idx"][rand_doc_idx][:-1]

        # rasgele tüm noktalara/sent_idx'lere gidilecek (without replacement şekilde)
        for rand_sent_start_idx in random.sample(doc_sent_idx_list, k=len(doc_sent_idx_list)):
            # rand_sent_start_idx + 1' deki +1 : nokta/sent_idx tokeni dahil etmemeliyiz B'ye, A sonunda hali hazırda nokta/sent_idx tokeni var
            # yoksa ab cümlelerinde A ve B arasında 2 tane nokta görürüz diye basitçe özetleyebiliriz
            if len(docs["token_ids"][rand_doc_idx][rand_sent_start_idx + 1:]) >= len_of_random_b:
                rand_sent_end_idx = rand_sent_start_idx + len_of_random_b
                random_b_dict["token_ids"] = docs["token_ids"][rand_doc_idx][rand_sent_start_idx + 1:rand_sent_end_idx + 1]
                random_b_dict["word_ids"] = docs["word_ids"][rand_doc_idx][rand_sent_start_idx + 1:rand_sent_end_idx + 1]
                return random_b_dict


    print(f"[INFO] Random sample couldn't be found in this shard interestingly...")
    return None


def _update_random_word_id(random_b_word_ids:List, last_word_id_of_A:int):
    # return updated_random_b_word_ids
    # 32  88
    temp = np.array(random_b_word_ids)
    
    if random_b_word_ids[0] > last_word_id_of_A:
        random_b_word_ids = temp - (random_b_word_ids[0] - last_word_id_of_A - 1)
    else:
        random_b_word_ids = temp + (last_word_id_of_A - random_b_word_ids[0] + 1)
    return random_b_word_ids.tolist()


def _fill_ab_from_block(ab_dict, doc, block_start_idx, mid_sent_idx, block_end_idx):
    ab_dict["A_token_ids"].append(doc["token_ids"][block_start_idx:mid_sent_idx])
    ab_dict["A_word_ids"].append(doc["word_ids"][block_start_idx:mid_sent_idx])
    ab_dict["B_token_ids"].append(doc["token_ids"][mid_sent_idx:block_end_idx])
    ab_dict["B_word_ids"].append(doc["word_ids"][mid_sent_idx:block_end_idx])
    ab_dict["isNext"].append(True)


def _fill_a_from_block_b_from_random(ab_dict, doc, docs, block_start_idx, mid_sent_idx, len_of_random_b):
    random_b = _get_random_sample(docs, len_of_random_b)
    if random_b is not None:
        ab_dict["A_token_ids"].append(doc["token_ids"][block_start_idx:mid_sent_idx])
        ab_dict["A_word_ids"].append(doc["word_ids"][block_start_idx:mid_sent_idx])
        ab_dict["B_token_ids"].append(random_b["token_ids"][:])
        ab_dict["B_word_ids"].append(_update_random_word_id(random_b["word_ids"][:], ab_dict["A_word_ids"][-1][-1])) # updt_rnd_b(list, int)
        ab_dict["isNext"].append(False)

    # eğer random sample "bir şekilde" bulunamadıysa (çok zor bir ihtimal) hiç birşey yapma


def convert_doc_to_ab(docs: pd.DataFrame | dict[str, list], doc_idx: int, block_size: int = BLOCK_SIZE, 
               overlap: int = OVERLAP, tampon:int = TAMPON)-> dict[str, list]:
    """doc(pd.Series): doc, token_ids, word_ids, sent_idx"""

    doc = docs.iloc[doc_idx]

    block_size_raw = block_size - 3     # del special tokens (cls, sep, sep)
    block_start_idx = 0
    block_end_idx = block_size_raw

    ab_dict = {"A_token_ids":[], "B_token_ids":[], "A_word_ids":[], "B_word_ids":[], "isNext": []}


    doc_len = len(doc["token_ids"])
    while block_end_idx <= (doc_len):

        # block içerisindeki sent indexleri al
        block_sent_idx = [sent_idx for sent_idx in doc["sent_idx"] if (block_start_idx + tampon) < sent_idx and (block_end_idx - tampon) > sent_idx]
        number_of_sent = len(block_sent_idx)


        # block içerisinde sent idx yok
        if number_of_sent == 0:

            # block konumu güncelle
            block_start_idx += overlap
            block_end_idx += overlap

            # bir sonraki iterasyona atla
            continue

         # tek sayı olduğunda
        if number_of_sent % 2 != 0:
            # +1 nedeni: nokta/sent_idx A'nın içinde olsun
            mid_sent_idx = block_sent_idx[number_of_sent // 2] + 1  
        # çift sayı olduğunda
        else:
            # a ve b noktalarından hangisi blocksize ortasına yakın ise o nokta (sent idx) mid_sent_idx olarak alınacak
            a = number_of_sent // 2
            b = a - 1
            # +1 nedeni: nokta/sent_idx A'nın içinde olsun
            # unutma: mid_sent_idx, nokta/sent_idx token'ını indexler! ve bu token A'ya dahil olacak (fill fonk'larda görebilirsin)
            mid_sent_idx = (block_sent_idx[a] + 1) if abs(block_sent_idx[a] - block_size_raw // 2) < abs(block_sent_idx[b] - block_size_raw // 2) else (block_sent_idx[b] + 1)

        if random.random() > 0.5:
            # + 1 şurdan geldi: A nokta/sentidx token'ı da alacağından 1 birim sağdan kaydırdık
            len_of_random_b = block_size_raw - (mid_sent_idx - block_start_idx)

            _fill_a_from_block_b_from_random(ab_dict, doc, docs, block_start_idx, mid_sent_idx, len_of_random_b)

            # block konum güncellemesi yapılmadığına dikkat
            continue 
        

        _fill_ab_from_block(ab_dict, doc, block_start_idx, mid_sent_idx, block_end_idx)

        block_start_idx += overlap
        block_end_idx += overlap
    
    # BURASI AÇIK OLDUĞUNDA FALSE SAYISI ~3000 DAHA FAZLA OLUYOR (DOC SAYISI KADAR). CLASS IMBALANCE'A SEBEP OLABİLİR. GERÇİ FOCAL LOSS KULLANABİLİRİZ...
    # if (block_start_idx + tampon) < doc_len:
    #     len_of_random_b = block_end_idx - doc_len  # A midpoint yani nokta/sent_idx tokenınıda içine alacağından bir sağ shiftledik
    #     # random b için istenen token sayısı tampondan az ise o bu ab sample adayı kullanılmayacak (B'de çok çok az sayıda token var demektir)
    #     if len_of_random_b < tampon:
    #        return ab_dict
    #     _fill_a_from_block_b_from_random(ab_dict, doc, docs, block_start_idx, doc_len, len_of_random_b)

    return ab_dict


In [19]:
tokenizer.decode((18, 2871))

'. ekim'

* GÜNCELLEME: doc sınır aşıldığında aşılan miktar kadar pad token demiştik ama buna gerek yok, kalan yer random sample ile doldurulsun. (doc sonu zaten bir cümle sonu gibi yani ayrıca bir problem yok)

* GÜNCELLEME2: ana sistem çalışmadan önce tek sentence'a sahip tüm doclarda o sentence A olsun B için random yapılsın

* UYARI: nokta derken sent idx, sent idx derkende aslında 0'ı da dahil etmeyi unutma!!! 0: doc başlangıcı, noktalar: cümle araları veya doc bitişi

* UYARI: block_size - 3 'ü yani special tokenlar için yer unutma, isNext'i unutma

* UYARI: block sınırlarında tampon olmalı: sent idx'ler yani noktalar  

* HATIRLATMA: random B alındıktan sonra B nin word id'lerini update etmeyi unutma

* sent idx ile + doc tokenlaştırılması alttaki adımlardan ayrı ve önce yapılmalı (öbür türlü random sample zamanı gelen doc'u tokenize etmek durumunda kalırız ve o doc kullanılamadan (ihtiyaçları karşılamadığı için) başka bir doca gene rasgele atlanabilir)

* doc boyunca block hareket edecek (haraket deltası : overlap ile, block pencere genişliği indexleri: start/end_idx'ler ile)

* block penceresi doc sınırı dışına çıktığı an bu iş biter, döngüden çıkılır(tam üst üste çakışmada gene klasik ne yapıyorsa onu yapmalı ayrı bir şey yok)

* block penceresi doc sınırını aşıp çıktığında pencere indexleri güncellenecek (bunların farkı 1 ise (çok zor bir ihtimal eğer overlap parametresi block_size -1 kadar ise böyle bir ihtimal var genede dikkat edeceğim) komple iş bitti)

* güncellenen block penceresi aslında gene aynı işlemleri uygulayacak (random sample olayı yapılmamalı yanlız! döngü içinde olsun kısaca) olaylar eğer fonksiyonlaştırılırsa temiz şekilde halledilir. (func call overheadi bilmiyorum ne olur ne biter), kalan token yeri kadar PAD tokeni eklemlenmeli!

* block penceresi içerisinde 2 cümle yok ise (yani 1 tane sent index bile yoksa) bir sonraki it'e atla. rasgele B alınacağı durumda iterasyon donmalı (pencere start/endleri     güncellemeden bir sonraki iterasyona jumpla). 

* pencere içerisindeki kaç tane nokta varsa (sent index) ortasını seç

* seçilen orta noktanın solu A, sağı B olacak (token id'ler, word idlerde slicelanacak)

* sent indexlerde 0'ı sayma, yani tek bir cümlelik metin tek bir sent idx var 

* doc'ta sadece tek bir cümle varsa çık. 

* rasgele durumda: bulunulan pencerede nokta yoksa bir sonraki it'e atla(buna gerek yok zaten normalde de başta bu kontrol edilip bir sonraki it'e gidiliyor), block size - pencere içinde "orta" nokta uzunluğu = aranacak genişlik. Rasgele bir doca gidilecek bu doc genişliği bundan az ise başka random bir doc'a git. Genişliği sağlayan random doc bulunduğunda Random doc içerisindeki noktlara kümesinden rasgele nokta seç "without replacement!" o noktan tüm sağın genişliğine bak, bizi memnun ediyorsa onu al (token id, word id) etmiyorsa tekrar random nokta seç (ama tekrar ediyorum "without raplacement!" yani aynı noktayı birdaha seçememeliyim...) UYARI: nokta derken sent idx, sent idx derkende aslında 0'ı da dahil etmeyi unutma!!! 0: doc başlangıcı, noktalar: cümle araları veya doc bitişi

* PAD token olayı yanlızce doc sınırından çıkıldığında var, random sample'da YOK (aranan genişliği tatmin edecek yani bloğu komple dolduracak random sample arıyoruz da ondan dolayı)

her process dinamik şekilde list[dict] genişletip döndürecek (manager ile)

lookup table olaylarında (ayrıca statik size) shared memory kullanılacak (daha hızlı, senkronizasyon sistemi ve dolaylı olarak processler arası haberleşme overhead'i olmayacak. her proc tak diye istediğine erişecek)

## STAGE2 MP

In [54]:
import json

def read_shard(shard_dir, shard_idx, return_type="pd"):

    if shard_dir.startswith("doc"):
        prefix = "doc"
    elif shard_dir.startswith("ab"):
        prefix = "ab"

    if shard_idx >= 0 and shard_idx < len(os.listdir(prefix + "_shards")):
        
        temp = pd.read_json(shard_dir + prefix + f"_shard_{shard_idx}.json",
                            orient="records",
                            lines=True,
                            encoding="utf-8")
        
        if return_type == "pd":
            return temp

        elif return_type == "dict":
            return temp.to_dict("list")

        else:
            print(f"[ERROR] Unvalid input for return type, must be 'pd' or 'dict'...")

    else:
        raise IndexError(f"shard_idx >= 0 and shard_idx < {len(os.listdir('doc_shards'))}, shard_idx you gave was: {shard_idx}")

def create_doc_shards(docs_df):

    global NUM_SHARDS_FOR_DOC
    global NUM_DOCS_PER_SHARD

    NUM_DOCS_PER_SHARD = len(docs_df) // NUM_SHARDS_FOR_DOC


    if len(docs_df) % NUM_SHARDS_FOR_DOC != 0:
        extra_shard_len = len(docs_df) % NUM_SHARDS_FOR_DOC
        NUM_SHARDS_FOR_DOC += 1
        

    os.makedirs("doc_shards", exist_ok=True)
    files = os.listdir("./doc_shards")


    if len(files) == NUM_SHARDS_FOR_DOC:
        print(f"[INFO] Expected number of shards are already created. Exiting...")
        return
    
    # fazlalık dosyaları (eğer olursa) sil (evet biraz brute force)
    if NUM_SHARDS_FOR_DOC < len(files):

        files_finded = set(files)
        shard_files_expected = {f"doc_shard_{shard_idx}.json" for shard_idx in range(NUM_SHARDS_FOR_DOC)}
        files_will_deleted = list(files_finded - shard_files_expected)
        if files_will_deleted:
            print(f"[INFO] These files are unexpected, so they are gonna be deleted: {files_will_deleted}")
            for file in files:
                os.remove("./doc_shards/" + file)


    print(f"[INFO] Total number of doc: {len(docs_df)}")
    print(f"[INFO] Each shard will have {NUM_DOCS_PER_SHARD} number of docs...")
    print(f"[INFO] Number of shard will be: {NUM_SHARDS_FOR_DOC}")
    if extra_shard_len:
        print(f"[INFO] Extra shard needed. Number of doc will be placed: {extra_shard_len}")

    
    df_start_idx = 0
    df_end_idx = NUM_DOCS_PER_SHARD


    with tqdm.tqdm(total=NUM_SHARDS_FOR_DOC, desc="Doc shards are being created") as pbar:
        for shard_idx in range(0, NUM_SHARDS_FOR_DOC):
            # last iteration (extra shard)
            if shard_idx == NUM_SHARDS_FOR_DOC - 1:
                df_end_idx = df_start_idx + extra_shard_len

                assert df_end_idx == len(docs_df)
                assert (df_end_idx - df_start_idx) == extra_shard_len

                docs_df.iloc[df_start_idx:df_end_idx].to_json("./doc_shards/" + f"doc_shard_{shard_idx}.json",
                                                          orient="records",
                                                          lines=True,
                                                          force_ascii=False)
                pbar.update()
                break

            docs_df.iloc[df_start_idx:df_end_idx].to_json("./doc_shards/" + f"doc_shard_{shard_idx}.json",
                                                          orient="records",
                                                          lines=True,
                                                          force_ascii=False)
            df_start_idx = df_end_idx
            df_end_idx += NUM_DOCS_PER_SHARD
            pbar.update()
            

In [51]:
len(os.listdir("doc_shards"))

101

In [56]:
read_shard("doc_shards/", 100)

,doc
0,Afrika ada ülkesi São Tomé ve Príncipe kendi i...
1,1971'de kurulan Statistics Canada (StatCan; Fr...
2,"Red Bay, Kanada'daki Newfoundland ve Labrador ..."
3,"Denzil Holles, ""Holles Baronu"" (31 Ekim 1599 -..."
4,"Beyaz Rusya Devlet Kontrol Komitesi (), Belaru..."
...,...
88,"Carol Hanisch (d. 1942 Iowa ABD), Amerikalı ra..."
89,"Mutasarrıf (), Osmanlı idari sisteminde sancak..."
90,Eş görevli sözcük ya da sözcük gruplarının ara...
91,"""C'est la chanson de mon amour"" (), Véronique ..."


In [30]:
def visualize_ab(ab: pd.Series):
    """visualize one ab sample"""

    print(f"A: {ab['A_token_ids']}")
    print(f"B: {ab['B_token_ids']}")
    print(f"A_decoded: {tokenizer.decode(ab['A_token_ids'])}")
    print(f"B_decoded: {tokenizer.decode(ab['B_token_ids'])}")
    print(f"len_of_A: {len(ab['A_token_ids'])}")
    print(f"len_of_B: {len(ab['B_token_ids'])}")
    print(f"A_word_ids: {ab['A_word_ids']}")
    print(f"B_word_ids: {ab['B_word_ids']}")
    print(f"len_of_A_word_ids: {len(ab['A_word_ids'])}")
    print(f"len_of_B_word_ids: {len(ab['B_word_ids'])}")
    print(f"sum_of_AB_tokens: {len(ab['B_word_ids']) + len(ab['A_word_ids'])}")
    print(f"isNext: {ab['isNext']}")
    
    print("---------------\n")

In [ ]:
def create_ab_shards():

    os.makedirs("ab_shards", exist_ok=True)

    for shard_idx in range(len(os.listdir("doc_shards"))):
        docs_shard_df = read_shard("doc_shards/", shard_idx)
    
        docs_shard_df["token_ids"] = None
        docs_shard_df["word_ids"] = None
        docs_shard_df["sent_idx"] = None
        
        for doc_idx in range(len(docs_df)):
            docs_df.iloc[doc_idx] = tokenize_and_sent_idx(docs_df.iloc[doc_idx])


        ab_df = pd.DataFrame({"A_token_ids":[], "B_token_ids":[], "A_word_ids":[], "B_word_ids":[], "isNext": []})

        for doc_idx in range(len(docs_df)):
            ab_df = pd.concat([pd.DataFrame(convert_doc_to_ab(docs_df, doc_idx)), ab_df])

        # shuffle
        ab_df = ab_df.sample(frac=1)
        


In [62]:
import pandas as pd

# Dict objelerinden oluşan bir liste, değerleri listeler
ax = [
    {'A': [1, 2], 'B': [3, 4]},
    {'A': [5, 6], 'B': [7, 8]},
    {'A': [9, 10], 'B': [11, 12]}
]

# DataFrame'e çevirirken, her bir dict'i düzleştir
df = pd.DataFrame(ax)

# DataFrame'i yazdır
print(df)

         A         B
0   [1, 2]    [3, 4]
1   [5, 6]    [7, 8]
2  [9, 10]  [11, 12]


# Data az olduğundan dolayı vocab arttırma iyi olabilir (ne kadar parçalar isek özellikle tr dili için)

In [1]:
from data import *


%load_ext autoreload
%autoreload 2

In [2]:
PAD_TOKEN_ID, SEP_TOKEN_ID, CLS_TOKEN_ID, UNK_TOKEN_ID, MASK_TOKEN_ID

(0, 3, 2, 1, 4)

In [3]:
ab_row = read_shard("ab_shards_256/", 100).iloc[0]

In [4]:
visualize_sample(ab_row)

A: [20, 28, 4826, 2908, 3962, 6256, 7339, 16767, 28683, 2168, 2023, 1941, 3928, 5518, 6723, 18, 2424, 5644, 2539, 2914, 13250, 6089, 16, 4215, 7528, 1007, 6442, 1933, 30556, 8360, 4883, 12285, 11, 2028, 7182, 4774, 2410, 2799, 16, 18783, 16345, 4625, 7182, 2640, 1011, 2287, 16, 2126, 2702, 1011, 2914, 6717, 2552, 18, 1989, 2319, 18813, 16781, 2159, 5191, 10276, 1008, 2799, 3228, 5313, 3513, 17180, 2141, 14016, 1012, 14855, 6432, 18]
B: [23, 20, 2900, 22, 20, 20, 29, 2432, 16767, 28683, 2168, 2098, 4122, 8569, 18, 22, 25, 2908, 22, 20, 20, 29, 2432, 2498, 3282, 4865, 21867, 11, 2760, 22, 15, 21, 15871, 13639, 4058, 18, 4865, 21867, 2023, 7044, 23, 25, 6838, 3474, 8028, 9321, 29, 16, 26, 23, 3019, 12983, 2124, 3637, 2464, 18, 22, 26, 2900, 22, 20, 21, 21, 2432, 3656, 12392, 2023, 1941, 3928, 5518, 6723, 18, 12392, 2023, 18326, 1005, 11, 1960, 3064, 13225, 7824, 28221, 16, 2397, 5273, 5178, 2079, 11357, 13561, 11, 41, 24, 17, 22, 2023, 14707, 7044, 2826, 8629, 18, 22, 20, 21, 22, 11, 1960

In [5]:
x, y, a, b, c = gecicici(ab_row, return_selected=True)
selected_groups = (a, b, c)

In [6]:
visualize_sample(gecicici(ab_row), selected_groups=selected_groups, no_id=True)

x_decoded: [CLS] giymiştir 8 yılının ağustos ayında nba takımlarından cleveland [MASK] [MASK] ile bir yıllık sözleşme imzaladı. sezon başlarında az süre almasına [MASK], delonte west ve sasha pavlović'in sakatlanmalarının ardından, orlando magic maçında sakatlanana kadar, daha faza süre almaya başladı. bu yazı etrafındaki sakatlıktan kurtulmalarının ardından tekrar takımın üçüncü şutör [MASK] [MASK] [MASK] düştü. 3 0 temmuz 2 0 0 9 tarihinde cleveland cavaliers tarafından serbest bırakıldı. 2 5 ağustos 2 0 0 9 tarihinde eski takımı fenerbahçe ülker'le 2 + 1 yıllığına anlaşmaya vardı. fenerbahçe ülker ile ligde [MASK] 5 maça çıkan [MASK] [MASK] 9, 6 3 sayı ortalamasıyla mücadele etti. 2 6 temmuz 2 0 1 1 tarihinde anadolu efes [MASK] bir [MASK] sözleşme imzaladı. efes ile tbl'de final oynama başarısı gösterdiler [MASK] ancak finalde beşikaş milangaz'a 4 - 2 ile kaybederek ligde [MASK] oldular. 2 0 1 2'de golden state warriors ile sözleşme imzaladı, ancak takımın son kadrosunda yer alamay

In [7]:
# Orijinal dizi
arr = np.array([1, 2, 3, 4, 5])

# Dizi dilimi oluştur
sliced_arr = arr[1:4]

# Orijinal diziyi sil
del arr

# Sliced diziye erişim
try:
    print(sliced_arr)  # Bu erişim hala geçerli ama veri kaybı olur
except NameError:
    print("Orijinal dizi silindi.")

[2 3 4]


In [16]:
import numpy as np

arr = np.array([1, 2, 3, 4, 5])

for i in range(len(arr) - 1, -1, -1):
    print(arr[i])

5
4
3
2
1


* kaç tane kelimem var?
* kelimlelerden numpy array oluştur 0-1 arasında olasılık verdir
* bu kelime olasılık array'inden kelimelerin tipini belirle (mask_kelimeleri, replace_kelimeleri, identiy_kelimeleri belirle):
    * işte spesifik bir olaslık türündeki kelimeleri alıp (condition ile value return ettir index değil!) o kelime list/belkide array'ine eklemdir

* her kelime grubu için kelimelerin kaç tokenlık old u belirlemen lazım (binary search left alg kullanılacak)
    * döngü içerisinde spesifik kelime grubundaki her kelime için birleştirilmiş word_id array'inde binarysearchleft ile o kelimenin başlangıç indexini bul. sağa doğru farklı kelimeyle karşılaşana kadar bitiş indexini arttır. böylece kelime penceresi/slice oluştu. Buradan kaç token bilgiside geleceğinden kalan olayları sen anladın zaten anlatmıyorum.


* ab shard'ları dataframe olarak yükleniyorlar
* ab shardlarında row row (1 sample/example) hareket edilecek: pd.Series
* y placeholder full pad token_id'li numpy array
* x 'te numpy array olsun
* başta A ve B nin uzunlukarını al (len fonksiyonu O(1)) sonra A ve B yi birleştir ki işlemlerin kolay olsun (Daha sonra ayırıp araya sep tokeni koyarız)

* special tokenları, isNext'i vs unutma

* veri setimde tüm blockları dolu tuttuğum için (pad tokenı içerilmiyor) bundan dolayı attention mask komple 1 olacak (model eğitiminde oluşturmak gereksiz işlem kaybı olur ondan burada oluşturalım)
* segment_ids, numpy array şeklinde olacak; 0: A, 1:B idi sanırsam bir bakarım


* shard dosya büyüklüğü model eğitiminde tatmin etmezse diye shard dosyalarını fuse'layan ayrı bir fonk yazabilirsin!.
* Tokenizer'ı Cased olarak kullanmak daha iyi olabilir nsp açısından
* FOCAL LOSS KULLANIRSAK EKSRA FALSE DATASIDA KULLANILABİLİR